<a href="https://colab.research.google.com/github/mmdrez4/NLP_bio_data_analysis/blob/main/MIR_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <strong><font color=#A52A2A></span> MIR_HW3 </strong>

98101566 -> Mohammadreza Daviran
<br>
98105919 -> Reza Erfan Arani
<br>
98106434 -> Mohammad Khodadadi Aski
<br>

<strong><font color=#B22222></span> Please run this notebook on google colab! </strong>

# <strong><font color=#5F9EA0></span> Download the files </strong> 

In [ ]:
import requests
import json

In [ ]:
!git clone https://github.com/language-ml/course-nlp-ir-1-text-exploring.git

fatal: destination path 'course-nlp-ir-1-text-exploring' already exists and is not an empty directory.


In [ ]:
%cd course-nlp-ir-1-text-exploring/exploring-datasets/health

/content/course-nlp-ir-1-text-exploring/exploring-datasets/health


In [ ]:
!ls

bio.json	 hidoctor-3.json  namnak-1.json  namnak-4.json	stopwords.txt
hidoctor-1.json  hidoctor-4.json  namnak-2.json  namnak-5.json
hidoctor-2.json  hidoctor-5.json  namnak-3.json  README.md


Alternative way to download the files:

In [ ]:
# files = []
# result = list()
# for i in range(1, 6):
#   link = "https://github.com/language-ml/course-nlp-ir-1-text-exploring/blob/main/exploring-datasets/health/hidoctor-" + str(i) + ".json"
#   hidoctor = requests.get(link, allow_redirects=True)
#   name = 'hidoctor-' + str(i) + '.json'
#   temp_file = open(name, 'wb').write(hidoctor.content)
#   files.append(name)


# for i in range(1, 6):
#   link = "https://github.com/language-ml/course-nlp-ir-1-text-exploring/blob/main/exploring-datasets/health/namnak-" + str(i) + ".json"
#   namnak = requests.get(link, allow_redirects=True)
#   name = 'namnak-' + str(i) + '.json'
#   temp_file = open(name, 'wb').write(namnak.content)
#   files.append(name)

Our dataset is json so first of all we merge all the files to 'bio.json' file.

In [ ]:
def merge_JsonFiles(filename):
    result = list()
    for f1 in filename:
        with open(f1, 'r') as infile:
            result.extend(json.load(infile))

    with open('bio.json', 'w') as output_file:
        json.dump(result, output_file)

files = ['hidoctor-1.json', 'hidoctor-2.json', 'hidoctor-3.json','hidoctor-4.json','hidoctor-5.json', 'namnak-1.json', 'namnak-2.json', 'namnak-3.json','namnak-4.json','namnak-5.json']
merge_JsonFiles(files)

By loading the json file, we extract all the paragraphs and save them in *documents* for future work.

In [ ]:
f = open('bio.json')
bioset = json.load(f)
f.close()

In [ ]:
documents = []
for i in bioset:
  documents.append(i['paragraphs'])

# <strong><font color=#5F9EA0></span> Preprocessing </strong> 

First we import the requirements for preprocessing the data.

In [ ]:
import random 
import tqdm
import nltk
from nltk.corpus import brown
from nltk import word_tokenize
import csv
import re
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import random 
import numpy as np
import nltk
import codecs
import tqdm

In [ ]:
!pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from __future__ import unicode_literals
from hazm import *
import nltk

**normalize**

normalizer function normilizes the docs we extracted and saves them 



In [ ]:
normalizer = Normalizer()

doc_normalized = [[normalizer.normalize(y) for y in x] for x in tqdm.tqdm(documents)]

In [ ]:
doc_normalized_edited = []
for doc in doc_normalized:
  temp = []
  for x in doc:
    temp.append(x.replace('\u200c', ' '))
  doc_normalized_edited.append(temp)

**wording**

here we tokenize the sentences in each normilized document and then we save them in the doc_sentences array 

In [ ]:
doc_sentences = [sent_tokenize(' '.join(x)) for x in tqdm.tqdm(doc_normalized_edited)]

100%|██████████| 2125/2125 [00:00<00:00, 6184.95it/s]


**tokenize**

we tokenize the words in the doc_sentences array and the we save them into doc_tokens array by tqdm tools.

In [ ]:
doc_tokens = [[word_tokenize(sent) for sent in sents] for sents in tqdm.tqdm(doc_sentences)]

100%|██████████| 2125/2125 [00:02<00:00, 869.13it/s]


Then by using itertools save them in all_tokens. Because we want to exclude the stopwords from our tokens.

In [ ]:
all_tokens = []
for i in doc_tokens:
  all_tokens.append(list(itertools.chain.from_iterable(i)))

**Delete stop words**

here we 


1.   download the persian stopwords from bellow link
2.   then we delete them from all_tokens arraylist cause they have no value for us
3. save the tokens which are not stopwords in all_tokens_nonstop arraylist




In [ ]:
import codecs
link = "https://raw.githubusercontent.com/sobhe/hazm/master/hazm/data/stopwords.dat"
temp_file = open('stopwords.txt', 'wb').write(requests.get(link, allow_redirects=True).content)

In [ ]:
stopwords = [normalizer.normalize(x.strip()) for x in codecs.open('stopwords.txt','r','utf-8').readlines()]

In [ ]:
all_tokens_nonstop = []
for doc in all_tokens:
  all_tokens_nonstop.append([t for t in tqdm.tqdm(doc) if t not in stopwords])

**lemmatization, stemming**

here we lemmatize and stem the tokens 

In [ ]:
stemmer = Stemmer()
lemmatizer = Lemmatizer()

def get_lemma_set(tok, opt=1):
    if opt == 1:
        return stemmer.stem(tok)
    if opt == 2:
        return lemmatizer.lemmatize(tok)

In [ ]:
opt = 2

all_tokens_nonstop_lemstem = []

for doc in all_tokens_nonstop:
  all_tokens_nonstop_lemstem.append([get_lemma_set(t, opt) for t in tqdm.tqdm(doc)])

# <strong><font color=#5F9EA0></span> Boolean retrieval </strong> 

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import sent_tokenize , word_tokenize
import glob
import re
import os
import numpy as np
import sys

all words longer than 2 characters are stored in bag_of_words array 
all unique words in the bag of words set are stored in unique_words

In [ ]:
bag_of_words = []
unique_words = set()
lemmatized_docs = [' '.join(x) for x in all_tokens_nonstop_lemstem]
for doc in lemmatized_docs:
  temp = doc.split(' ')
  bag_of_word = [t for t in temp if len(t) > 2]
  bag_of_words.append(bag_of_word)
  unique_words = set(unique_words).union(set(bag_of_word))

we specify which words from unique_words each document contains and store their number in num_of_words.

In [ ]:
num_of_words = []

for i in range(len(bag_of_words)):
  numOfWord = dict.fromkeys(unique_words, 0)
  for word in bag_of_words[i]:
    numOfWord[word] += 1
  num_of_words.append(numOfWord)

We illustrate the num_of_words list for the first 10 documents for better comprehension:

In [ ]:
df = pd.DataFrame(num_of_words[0:10])
df

,اشتهای,عفونی,اکسون,_گرسنگی,صیه,خواهند_مصرف,شادکامی,غذاخورده,Jose,کاهو,...,دیمتیل,بیوتکنولوژیک,اوروفارنکس,وافل,کلانژیت,بیهوشی,آلکسی,Konsyl,بولدو,B۲و
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


First we normalize and lemmatize our query.

In [ ]:
input = 'علل ضعف بینایی'
splitted_input = input.split(" ")
nsi = []
k = 12
for x in splitted_input:
  if (x not in stopwords and len(x) > 2 ):
    nsi.append(normalizer.normalize(lemmatizer.lemmatize(x))) 


To compare the query with our documents, we just need to find out in which documents the words of our query are. So we assign score for each doc with respect to the existence of words and the number of them. 

<strong><font color=#cf6679></span> We consider the number of words, so our implemented method is better than the ordinary Boolean method. If we want to implement ordinary boolean method we simply can comment one line!</strong> 

if you want to make the algorithm better , you can assume a negative point for not having a word

In [ ]:
score = {}
for i in range(len(all_tokens_nonstop_lemstem)):
  score[i] = 0

counter = 0
for doc in num_of_words:
  for word in nsi:
    try :
      if doc[word] > 0:
        score[counter] += 1
        # you can comment the next line
        score[counter] += 0.02 * doc[word]
    except :
      pass
  counter += 1
sorted_score = {k: v for k, v in sorted(score.items(), key=lambda item: item[1], reverse=True)}

In [ ]:
counter = 0
for x in sorted_score:
  if counter >= k:
    break
  print(bioset[x]['title'])
  print(bioset[x]['link'])
  print()
  counter += 1

آتروفی یا تحلیل عضلات: علل، علائم، پیشگیری و درمان
https://www.hidoctor.ir/328173_%d8%a2%d8%aa%d8%b1%d9%88%d9%81%db%8c-%db%8c%d8%a7-%d8%aa%d8%ad%d9%84%db%8c%d9%84-%d8%b9%d8%b6%d9%84%d8%a7%d8%aa.html/

اطلاعات جامع در مورد بیماری منییر
https://www.hidoctor.ir/276282_%d8%a7%d8%b7%d9%84%d8%a7%d8%b9%d8%a7%d8%aa-%d8%ac%d8%a7%d9%85%d8%b9-%d8%af%d8%b1-%d9%85%d9%88%d8%b1%d8%af-%d8%a8%db%8c%d9%85%d8%a7%d8%b1%db%8c-%d9%85%d9%86%db%8c%db%8c%d8%b1.html/

هورمون رشد خوبه یا بد؟
https://namnak.com/هورمون-رشد.p33051

اطلاعاتی در مورد سرطان و دیابت
https://www.hidoctor.ir/275852_%d8%a7%d8%b7%d9%84%d8%a7%d8%b9%d8%a7%d8%aa%db%8c-%d8%af%d8%b1-%d9%85%d9%88%d8%b1%d8%af-%d8%b3%d8%b1%d8%b7%d8%a7%d9%86-%d9%88-%d8%af%db%8c%d8%a7%d8%a8%d8%aa.html/

سرمه کشیدن و این همه خاصیت برای سلامت چشم
https://www.hidoctor.ir/285564_%d8%b3%d8%b1%d9%85%d9%87-%da%a9%d8%b4%db%8c%d8%af%d9%86-%d9%88-%d8%a7%db%8c%d9%86-%d9%87%d9%85%d9%87-%d8%ae%d8%a7%d8%b5%db%8c%d8%aa-%d8%a8%d8%b1%d8%a7%db%8c-%d8%b3%d9%84%d8%a7%d9%85%d8%aa-%da%86

**Alternative solution**

we can do all these things and findout the scores without num_of_words.
here is alternative solution for this problem:

In [ ]:
# input = input("Enter your subjects :")
input = 'ارتباط حمله قلبی و کرونا'

In [ ]:
score = []
def reset_score():
  for i in range(len(all_tokens_nonstop_lemstem)):
    score.append(0)

In [ ]:
splitted_input = input.split(" ")
nsi = []
for x in splitted_input:
  if x not in stopwords:
    nsi.append(normalizer.normalize(lemmatizer.lemmatize(x))) 
reset_score()
counter = 0
for doc in all_tokens_nonstop_lemstem :
  for x in nsi:
    if x in doc:
      score[counter]+=1
  counter += 1
print(score)

# <strong><font color=#5F9EA0></span> tf−idf </strong> 

Download all the requirements especially TfidfVectorizer from sklearn.

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

We use TfidfVectorizer with (1, 2) ngram and norm l2 for our vectorizer.
Then we find all the documents' vectors with fit_transform.

In [ ]:
vectorizer = TfidfVectorizer(use_idf = True, norm ='l2', ngram_range=(1,2), analyzer='word')
doc_term_mat = vectorizer.fit_transform([' '.join(doc) for doc in all_tokens_nonstop_lemstem])
vocabulary = vectorizer.get_feature_names_out()

In this function we enter the query and do all the preprocessing on that.

In [ ]:
def process_query(query):
  splitted_input = query.split(" ")
  nsi = []
  for x in splitted_input:
    if x not in stopwords:
      nsi.append([normalizer.normalize(lemmatizer.lemmatize(x))])
  return nsi

To compare the query with our documents we need to find query_vector.

In [ ]:
def find_query_vector(tokens):
  vector = np.zeros(len(vocabulary))
  for token in itertools.chain(*tokens):
      try:
          index = vectorizer.vocabulary_[token]
          vector[index] = 1
      except ValueError:
          pass
  return vector

After calculating query_vector, we need To see how close one doc is to the query. So we find the cosine distant between query_vector and each doc. Then we sort the indexes according to the scores.

In [ ]:
def search(query, k):
  scores = []
  tokens = process_query(query)
  query_vector = find_query_vector(tokens)

  for doc in doc_term_mat.A:
    scores.append(np.dot(query_vector, doc)/(np.linalg.norm(query_vector)*np.linalg.norm(doc)))
  return np.array(scores).argsort()[-k:][::-1]

In [ ]:
query = "نشانه های سکته مغزی"
x = search(query, k = 12)
for index in x:
  print(bioset[index]['title'])
  print(bioset[index]['link'])
  print()

# <strong><font color=#5F9EA0></span> Transformer </strong> 

In this section we use trasformer models to embe

In [ ]:
!pip install transformers faiss torch

In [ ]:
from transformers import AutoConfig, AutoTokenizer, AutoModel, TFAutoModel, BigBirdModel
import torch

Check if gpu is available or not:

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

There are not many Persian models for Transformers. 

For this reason, we use one of the two models, **Parsbert** or **ParsBigBird**.

In [ ]:
choice = 2

if choice == 1:
  model_name_or_path = "HooshvareLab/bert-fa-zwnj-base"
  config = AutoConfig.from_pretrained(model_name_or_path)
  tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

  # model = TFAutoModel.from_pretrained(model_name_or_path).to(device)
  model = AutoModel.from_pretrained(model_name_or_path).to(device)

if choice == 2:

  MODEL_NAME = "SajjadAyoubi/distil-bigbird-fa-zwnj"

  # model = BigBirdModel.from_pretrained(MODEL_NAME, block_size=32).to(device)
  model = BigBirdModel.from_pretrained(MODEL_NAME, attention_type="original_full").to(device)
  tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

print(model.device)

Use preprocessed data and join them together for further works:

In [ ]:
transformer_documents = [' '.join(x) for x in all_tokens_nonstop_lemstem]

Unfortunalely available models for transformers can not process documents with more than 512 characters. so we partition each document to sections with 512 or less characters. Afterwards, If our data length exceeds the specified limit, partition_doc function find their embeddings and calculate the average of the vectors as the final embeddeding for each document recursively. Otherwise we calculate the embedding simply by the model.

In [ ]:
def partition_doc(document):
  x = []
  def partition(document):
    if len(document) == 0:
      return x
    t = tokenizer(document[:512], return_tensors='pt').to(device)
    m = model(**t)[0].detach().squeeze()
    x.append(torch.mean(m, dim=0))
    return partition(document[512:])
  z = partition(document)
  return z

In this loop we tokenize the document and return it as PyTorch tensors and pass it onto the model:


In [ ]:
averaged_vectors = []
counter = 0 
for document in transformer_documents:
  try:
    x = tokenizer(document, return_tensors='pt').to(device)
    m = model(**x)[0].detach().squeeze()
    averaged_vectors.append(torch.mean(m, dim=0))
  except:
    tensors = partition_doc(document)
    result = torch.stack(tensors, dim=0).sum(dim=0)
    averaged_vectors.append(torch.div(result, len(tensors)))
  if counter % 100 == 0:
    print(counter)
  counter +=1

Token indices sequence length is longer than the specified maximum sequence length for this model (1393 > 512). Running this sequence through the model will result in indexing errors


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100


In [ ]:
[v.size() for v in averaged_vectors[:5]]

[torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768])]

In [ ]:
x = tokenizer(transformer_documents[1], return_tensors='pt').to(device)
m = model(**x)[0].detach().squeeze()
m

tensor([[-0.7518, -0.2972,  0.5164,  ...,  0.9054, -1.0787, -0.7217],
        [ 0.9634, -0.1464, -0.5599,  ...,  0.0650,  1.0207,  0.7631],
        [-0.7309,  0.3546, -0.2604,  ...,  0.0146,  1.0128, -0.4903],
        ...,
        [ 0.4148, -0.3143, -0.1375,  ...,  0.5281,  0.2779, -1.3112],
        [ 0.7779, -1.2898,  0.0992,  ...,  0.3979,  0.1694, -1.8513],
        [ 1.7691, -0.2985,  0.4894,  ...,  0.8071,  0.4125,  0.0774]],
       device='cuda:0')

We install the requirements for using faiss and query and searching!

In [ ]:
!apt-get install libomp-dev

In [ ]:
import faiss
import numpy as np

Index all the documents, we need them as numpy arrays first


In [ ]:
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
index.add_with_ids(np.array([t.cpu().numpy() for t in averaged_vectors]), np.array(range(0, len(documents))))

With *search* and *encoder* function we can encode queries into the vector space and find the most relevant documents to them with the help of faiss. Also we calculate the score of each document and return them with respect to ***k***.

In [ ]:
def encoder(document: str) -> torch.Tensor:
  tokens = tokenizer(document, return_tensors='pt').to(device)
  vector = model(**tokens)[0].detach().squeeze()
  return torch.mean(vector, dim=0)

In [ ]:
def search(query: str, k=12):
  encoded_query = encoder(query).unsqueeze(dim=0).cpu().numpy()
  top_k = index.search(encoded_query, k)
  scores = top_k[0][0]
  return top_k[1][0]

In this section we can enter our input and after normalization and lemmatization of the query, we return top ***k*** relevant documents titles and links.

In [ ]:
input = 'نقش استرس در حمله قلبی'
splitted_input = input.split(" ")
nsi = []
k = 12
for x in splitted_input:
  if x not in stopwords:
    nsi.append(normalizer.normalize(lemmatizer.lemmatize(x))) 

input = " ".join(nsi)

In [ ]:
res = search(input, k)
res

array([ 438, 1017, 1169, 1917,  184,  932, 1370, 1427, 1619, 1014,  267,
       1516])

In [ ]:
for x in res:
  print(bioset[x]['title'])
  print(bioset[x]['link'])
  print()

ناشتای سالم برای افراد سالم
https://www.hidoctor.ir/36118_%d9%86%d8%a7%d8%b4%d8%aa%d8%a7%db%8c-%d8%b3%d8%a7%d9%84%d9%85-%d8%a8%d8%b1%d8%a7%db%8c-%d8%a7%d9%81%d8%b1%d8%a7%d8%af-%d8%b3%d8%a7%d9%84%d9%85.html/

سودمندی ویتامین دی در بهبود كمردرد
https://www.hidoctor.ir/1756_%d8%b3%d9%88%d8%af%d9%85%d9%86%d8%af%db%8c-%d9%88%db%8c%d8%aa%d8%a7%d9%85%db%8c%d9%86-%d8%af%db%8c-%d8%af%d8%b1-%d8%a8%d9%87%d8%a8%d9%88%d8%af-%d9%83%d9%85%d8%b1%d8%af%d8%b1%d8%af.html/

غذا خوردن به همراه دوستان و آشنایان
https://www.hidoctor.ir/137551_%d8%ba%d8%b0%d8%a7-%d8%ae%d9%88%d8%b1%d8%af%d9%86-%d8%a8%d9%87-%d9%87%d9%85%d8%b1%d8%a7%d9%87-%d8%af%d9%88%d8%b3%d8%aa%d8%a7%d9%86-%d9%88-%d8%a2%d8%b4%d9%86%d8%a7%db%8c%d8%a7%d9%86.html/

ورزش کردن راهکاری موثر برای پیشگیری از آلزایمر
https://namnak.com/پیشگیری-از-آلزایمر.p49776

ارتباط بین تغذیه و سلامتی در زندگی انسان
https://www.hidoctor.ir/125875_%d8%a7%d8%b1%d8%aa%d8%a8%d8%a7%d8%b7-%d8%a8%db%8c%d9%86-%d8%aa%d8%ba%d8%b0%db%8c%d9%87-%d9%88-%d8%b3%d9%84%d8%a7%d9%85%d8

# <strong><font color=#5F9EA0></span> fasttext </strong> 

ّFirst we need to download, extract and load Fasttext word embedding model to use it in our code: 

*   Trained persian language dataset
*   fasttext library

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fa.300.bin.gz

In [ ]:
# !gunzip ../content/cc.fa.300.bin.gz
!pip install fasttext
# import fasttext 

# model = fasttext.load_model("/content/cc.fa.300.bin")

In [ ]:
import fasttext.util
fasttext.util.download_model('fa', if_exists='ignore')  # Persian
ft = fasttext.load_model('cc.fa.300.bin')

We need numpy to calculate vector calculations.

In [ ]:
import numpy as np

Now you need to Enter a Query that we with Fasttext method later find the best results. This query first will be pre-processed and then the vectors based on our trained dataset will be built.

In [ ]:
print("Search: ", end='')
search = input()
search_words_splited = search.split(" ")
vectors = []

search_words = []
for x in search_words_splited:
  if x not in stopwords:
    search_words.append([normalizer.normalize(lemmatizer.lemmatize(x))])

for word in search_words:
  vectors.append(np.array(ft.get_word_vector(word)))

vectors = np.array(vectors)
search_vector = np.mean(vectors, axis=0)

To compare the query with our documents we will also need these docs' vectors.<br>
So in this section all of the doc's vectors will be build.

In [ ]:
doc_vectors = []
for i in range(len(all_tokens_nonstop)):
  vectors = []
  for word in all_tokens_nonstop[i]:
    vectors.append(np.array(ft.get_word_vector(word)))
  vectors = np.array(vectors)
  doc_vectors.append(np.mean(vectors, axis=0))

To see how close one doc is to the query we Find the cosine distant between Query-vector and each Doc-vectors.
<br>
<strong><font color=#cf6679>Also</span> "k" is set to be 10 you can change it yourself</strong> 

In [ ]:
k = 10

In [ ]:
distance_list = []

for doc_vector in doc_vectors:
  distance_list.append(np.dot(search_vector, doc_vector)/(np.linalg.norm(search_vector)*np.linalg.norm(doc_vector)))

Now we sort the distant list and find the best matches.

In [ ]:
sorted_dist_list = []
for doc_vec in distance_list:
  sorted_dist_list.append(doc_vec)
sorted_dist_list.sort(reverse=True)
doc_indices = []
for i in range(k):
  doc_indices.append(distance_list.index(sorted_dist_list[i]))

Printing the information of best matches.

In [ ]:
print(doc_indices)
for i in range(k):
  print(bioset[doc_indices[i]]['title'])
  print(bioset[doc_indices[i]]['link'])
  print()

# <strong><font color=#5F9EA0></span> Comparison between methods </strong> 

**According to the MRR rank:**

The best method is **tf-idf**. It has performed better than other methods in terms of both mean scores and opinions. The **transformer** method was expected to be better than the other methods, but due to the fact that the processed language is Persian, the existing models for Persian transformers perform much weaker than English, and there were limitations in using them for this volume. Also, in general, the **fasttext** method works better than the regular **boolean** method, but here, by modifying the **boolean** method in our code and paying attention to the number of words in addition to their presence or absence, we made these two methods work almost identically, which shows the effect of our work and the changes which we applied to the **boolean** method.

So in conclusion in our project the following relationship is established in terms of the efficiency of the methods:

**tf-idf** > **modified boolean** = **fasttext** > **boolean** > **transformer**